# This is a demo for sfc project
Author: Martin Ptacek
Project variant: Predikce řešená dopřednou backprop. NN
Recurrent networks, 2/29. - Tapped delay line forward feed neural net or TDNN

In [ ]:
import numpy as np
import pandas as pd

In [7]:
# load dataset
path = 'dataset_slim.csv'
df = pd.read_csv(path)
print(df.head())

              DateTime  Consumption
0  2019-01-01 00:00:00         6352
1  2019-01-01 01:00:00         6116
2  2019-01-01 02:00:00         5873
3  2019-01-01 03:00:00         5682
4  2019-01-01 04:00:00         5557


In [ ]:
# Check for missing values in the data
df.isnull().sum()

DateTime       0
Consumption    0
dtype: int64

In [9]:
# Check for anomalies
df.describe()

,Consumption
count,46011.000000
mean,6587.616440
std,1043.654923
min,3889.000000
25%,5773.000000
50%,6552.000000
75%,7321.000000
max,9615.000000


In [ ]:
# Get numpy array and normalize it
array = df["Consumption"].to_numpy()

# Z-score normalization
std = np.std(array)
mean = np.mean(array)
normalized_array = (array - mean) / std

In [ ]:
# Parameters
delay_line_length = 100

array([-0.22576332, -0.45189416, -0.68473227, ..., -0.57166685,
       -1.01626308, -1.41486659])

In [ ]:
# model

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

class TDNN(object):
    def foward(self, x):
        return np.sum(self.weights * x)